In [22]:
import pandas as pd
import numpy as np
import requests
import os
import shutil
from bs4 import BeautifulSoup
from selenium import webdriver

In [23]:
years = list(range(1995,2022))

## MVP

In [3]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = url_start.format(year)
    
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [4]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Year"] = year
    dfs.append(mvp_df)

In [5]:
mvps = pd.concat(dfs)

mvps

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,David Robinson,29,SAS,73.0,901.0,1050,0.858,81,38.0,...,10.8,2.9,1.7,3.2,0.530,0.300,0.774,17.5,0.273,1995
1,2,Shaquille O'Neal,22,ORL,12.0,605.0,1050,0.576,79,37.0,...,11.4,2.7,0.9,2.4,0.583,0.000,0.533,14.0,0.230,1995
2,3,Karl Malone,31,UTA,14.0,532.0,1050,0.507,82,38.1,...,10.6,3.5,1.6,1.0,0.536,0.268,0.742,13.8,0.212,1995
3,4,Patrick Ewing,32,NYK,2.0,230.0,1050,0.219,79,37.0,...,11.0,2.7,0.9,2.0,0.503,0.286,0.750,9.6,0.157,1995
4,5,Hakeem Olajuwon,32,HOU,1.0,147.0,1050,0.140,72,39.6,...,10.8,3.5,1.8,3.4,0.517,0.188,0.756,10.7,0.181,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,11,Russell Westbrook,32,WAS,0.0,5.0,1010,0.005,65,36.4,...,11.5,11.7,1.4,0.4,0.439,0.315,0.656,3.7,0.075,2021
11,12,Ben Simmons,24,PHI,0.0,3.0,1010,0.003,58,32.4,...,7.2,6.9,1.6,0.6,0.557,0.300,0.613,6.0,0.153,2021
12,13T,James Harden,31,TOT,0.0,1.0,1010,0.001,44,36.6,...,7.9,10.8,1.2,0.8,0.466,0.362,0.861,7.0,0.208,2021
13,13T,LeBron James,36,LAL,0.0,1.0,1010,0.001,45,33.4,...,7.7,7.8,1.1,0.6,0.513,0.365,0.698,5.6,0.179,2021


In [7]:
mvps.to_csv("mvp.csv")

## Teams

In [8]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [9]:
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("Standings/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [10]:
dataframes = []
for year in years:
    with open("Standings/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    dataframes.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dataframes.append(w_df)

In [11]:
teams = pd.concat(dataframes)

In [12]:
teams.tail(30)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
5,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,2021,Central Division
6,46,26,.639,—,120.1,114.2,5.57,2021,Milwaukee Bucks*
7,34,38,.472,12.0,115.3,115.3,-0.13,2021,Indiana Pacers
8,31,41,.431,15.0,110.7,111.6,-0.94,2021,Chicago Bulls
9,22,50,.306,24.0,103.8,112.3,-8.19,2021,Cleveland Cavaliers
10,20,52,.278,26.0,106.6,111.1,-4.38,2021,Detroit Pistons
11,Southeast Division,Southeast Division,Southeast Division,Southeast Division,Southeast Division,Southeast Division,Southeast Division,2021,Southeast Division
12,41,31,.569,—,113.7,111.4,2.14,2021,Atlanta Hawks*
13,40,32,.556,1.0,108.1,108.0,-0.06,2021,Miami Heat*
14,34,38,.472,7.0,116.6,118.5,-1.85,2021,Washington Wizards*


In [13]:
teams.to_csv("teams.csv")

## Players

In [14]:
url_2 = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'

In [15]:
players = list(range(1995, 2023))

In [16]:
for player in players:
    url_2_final = url_2.format(player)
    data = requests.get(url_2_final)
    
    with open ('Players/players{}.html'.format(player), 'w+') as file:
        file.write(data.text)

In [17]:
dataframes = []
for player in players:
    with open("Players/players{}.html".format(player)) as f:
        page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    players_list = soup.find_all(id="per_game_stats")[0]
    players_list_final = pd.read_html(str(players_list))[0]
    players_list_final["Year"] = player
    dataframes.append(players_list_final)

In [18]:
players = pd.concat(dataframes)

In [19]:
players

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Alaa Abdelnaby,PF,26,TOT,54,0,9.4,2.2,4.3,...,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,1995
1,1,Alaa Abdelnaby,PF,26,SAC,51,0,9.3,2.3,4.3,...,0.7,1.4,2.1,0.3,0.3,0.2,0.8,2.0,5.0,1995
2,1,Alaa Abdelnaby,PF,26,PHI,3,0,10.0,0.3,3.7,...,1.0,1.7,2.7,0.0,0.0,0.0,1.7,0.7,0.7,1995
3,2,Mahmoud Abdul-Rauf,PG,25,DEN,73,43,28.5,6.5,13.8,...,0.4,1.4,1.9,3.6,1.1,0.1,1.6,1.7,16.0,1995
4,3,Michael Adams,PG,32,CHH,29,0,15.3,2.3,5.1,...,0.2,0.8,1.0,3.3,0.8,0.0,0.9,1.4,6.5,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,601,Thaddeus Young,PF,33,TOR,26,0,18.3,2.6,5.5,...,1.5,2.9,4.4,1.7,1.2,0.4,0.8,1.7,6.3,2022
837,602,Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,...,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,2022
838,603,Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,...,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3,2022
839,604,Cody Zeller,C,29,POR,27,0,13.1,1.9,3.3,...,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2,2022


In [20]:
players.to_csv("players.csv")